# Let's build a Spam Detector

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv('SMSSpamCollection',
                      inferSchema=True,
                      sep='\t')

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
# Rename columns (data had no header...)
data = data.withColumnRenamed('_c0','class')
data = data.withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
# Add length of each text message
data = data.withColumn('length',length(data['text']))

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
# Interesting... Spam messages tend to be bigger
# Length is a relevant feature
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [11]:
# Now let's jump into "NLP feature engineering" more features
# Tokenizing
# Removing Stop Words
# Count Vectorizing
# Inverse Document Frequency
from pyspark.ml.feature import (Tokenizer, StopWordsRemover,
                                CountVectorizer, IDF,
                                StringIndexer)

In [14]:
# 1. Split text into words
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')

# 2. Remove "meaningless" words
stop_remove = StopWordsRemover(inputCol='token_text',
                               outputCol='stop_token')

# 3. Identify words as terms and output the count vector feature
count_vec = CountVectorizer(inputCol='stop_token',
                            outputCol='c_vec')

# 4. Grab term frequency feature and compute for ML
idf = IDF(inputCol='c_vec',outputCol='tf_idf')

# 5. Index label as category for ML
ham_spam_to_numeric = StringIndexer(inputCol='class',
                                    outputCol='label')

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
# Compute both features
cleanUp = VectorAssembler(inputCols=['tf_idf','length'],
                          outputCol='features')

In [17]:
# The classic classification model to use for NLP
# NAIVE BAYES
from pyspark.ml.classification import NaiveBayes

In [18]:
# Initiate model
nb = NaiveBayes()

In [19]:
from pyspark.ml import Pipeline

In [20]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,
                                  tokenizer,stop_remove,
                                  count_vec,idf,cleanUp])

In [21]:
cleaner = data_prep_pipe.fit(data)

In [22]:
clean_data = cleaner.transform(data)

In [24]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [25]:
clean_data = clean_data.select(['label','features'])

In [26]:
# Data is ready for ML
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [27]:
# Split data
training,test = clean_data.randomSplit([0.7,0.3])

In [28]:
# Fit to training data
spam_detector = nb.fit(training)

In [29]:
# View results on test data
results = spam_detector.transform(test)

In [31]:
results.select(['label','prediction']).show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 20 rows



In [32]:
# EVALUATE YOUR MODEL
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
acc_eval = MulticlassClassificationEvaluator()

In [34]:
acc = acc_eval.evaluate(results)

In [36]:
print('Accuracy of Naive Bayes Model')
print(acc)

Accuracy of Naive Bayes Model
0.9208371583661229
